In [14]:
import os, sys, math, pickle, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from random import randint as ri
from random import uniform as ru

from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.losses import Huber
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error

### Funciones

In [3]:
#Function to split the training data into X,y datasets
def load_dataset(scene_in = "modular02a"):
    # Defining usefull variables
    path = os.getcwd()
    file_list = os.listdir(path)
    scene_files_list = [item for item in file_list if scene_in in item and "pkl" in item]


    file = open(path + "\\" + scene_files_list[0], "rb")
    training_data = pickle.load(file)
    training_df = pd.DataFrame(training_data)

    #List to store the name for every joint data column
    increments_columns = []
    prev_j_positions_columns = []
    post_j_positions_columns = []

    #Creates the name for every column
    num_joints = len(training_data[-1]["increments"])
    for joint_n in range(num_joints):
        #List to split inputs per joint
        joint_inc_col_name = "increments_" + str(joint_n)
        increments_columns.append(joint_inc_col_name)

        prev_joint_pos_col_name = "prev_j_positions_" + str(joint_n)
        prev_j_positions_columns.append(prev_joint_pos_col_name)

        #List to split outputs per joint
        post_joint_pos_col_name = "post_j_positions_" + str(joint_n)
        post_j_positions_columns.append(post_joint_pos_col_name)


    #Input columns per joint
    increments_df = pd.DataFrame(training_df['increments'].to_list())
    increments_df.columns = increments_columns


    prev_j_positions_df = pd.DataFrame(training_df['prev_j_positions'].to_list())
    prev_j_positions_df.columns = prev_j_positions_columns


    #Builds the X dataframe
    X_df = pd.concat([increments_df, prev_j_positions_df, 
                    training_df["prev_pos_x"], training_df["prev_pos_y"], training_df["prev_pos_z"]], 
                    axis="columns")


    #Output columns per joint
    post_j_positions_df = pd.DataFrame(training_df['post_j_positions'].to_list())
    post_j_positions_df.columns = post_j_positions_columns


    #Builds the y dataframe
    y_df = pd.concat([post_j_positions_df, 
                    training_df["post_pos_x"], training_df["post_pos_y"], training_df["post_pos_z"]], 
                    axis="columns")

    return(X_df,y_df)

In [4]:
def adapt_X_y(X_a,y_a):
    X_a_cols = ['current_state_x', 'current_state_y', 'current_state_z', 'joint0_current_state_rad', 'joint1_current_state_rad', 'joint0_actions_rad', 'joint1_actions_rad']

    X_a = pd.concat([X_a["prev_pos_x"], X_a["prev_pos_y"] , X_a["prev_pos_z"],
                X_a["prev_j_positions_0"], X_a["prev_j_positions_1"],
                X_a["increments_0"], X_a["increments_1"]], 
                        axis="columns")

    X_a.columns = X_a_cols

    y_a_cols = ['future_state_x', 'future_state_y', 'future_state_z', 'joint0_future_state_rad', 'joint1_future_state_rad']

    y_a = pd.concat([y_a["post_pos_x"], y_a["post_pos_y"] , y_a["post_pos_z"],
                y_a["post_j_positions_0"], y_a["post_j_positions_1"]], 
                        axis="columns")
    y_a.columns = y_a_cols
    return X_a, y_a

### Modular 02

In [6]:
scene = "modular02_"
X, y =load_dataset(scene)
X, y = adapt_X_y(X,y)

In [7]:
# Se separan los datos de entrenmiento (80%) y los de prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2, random_state=42
)

In [8]:
# Scaling de los datos 
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
# Creacion y entrenamiento del modelo
model = Sequential()

# Capa de entrada con 22 unidades (correspondiente al número de entradas)
model.add(Dense(units=7, input_dim=7, activation='relu'))

# Capas ocultas con 64 unidades cada una
model.add(Dense(units=24, activation='relu'))

# Capa de salida con 14 unidades (correspondiente al número de salidas) y activación lineal
model.add(Dense(units=5, activation='linear'))

# Compilar el modelo con una función de pérdida adecuada para regresión
model.compile(optimizer=Adam(learning_rate=0.001), loss=Huber(delta=1.0))

# Supongamos que X_train y y_train son tus datos de entrenamiento
history = model.fit(X_train_scaled, y_train, epochs=25, batch_size=32, validation_split=0.2)

Epoch 1/25
700/700 [==============================] - 2s 2ms/step - loss: 0.0268 - val_loss: 0.0017
Epoch 2/25
700/700 [==============================] - 1s 1ms/step - loss: 8.9532e-04 - val_loss: 3.8412e-04
Epoch 3/25
700/700 [==============================] - 1s 1ms/step - loss: 2.5123e-04 - val_loss: 1.6989e-04
Epoch 4/25
700/700 [==============================] - 1s 1ms/step - loss: 1.4219e-04 - val_loss: 1.1901e-04
Epoch 5/25
700/700 [==============================] - 1s 1ms/step - loss: 1.0752e-04 - val_loss: 1.0169e-04
Epoch 6/25
700/700 [==============================] - 1s 1ms/step - loss: 8.9917e-05 - val_loss: 8.6052e-05
Epoch 7/25
700/700 [==============================] - 1s 1ms/step - loss: 7.7599e-05 - val_loss: 6.5175e-05
Epoch 8/25
700/700 [==============================] - 1s 1ms/step - loss: 6.7622e-05 - val_loss: 8.1524e-05
Epoch 9/25
700/700 [==============================] - 1s 1ms/step - loss: 6.2695e-05 - val_loss: 6.2157e-05
Epoch 10/25
700/700 [===============

In [11]:
model.save("model_" + scene + ".keras")
with open("scaler_" + scene + '.pkl', 'wb') as file:
    pickle.dump(scaler, file)
with open("model_" + scene + '.pkl', 'wb') as file:
    pickle.dump(model, file)

### Pruebas

In [25]:
#Function to split the training data into X,y datasets
def load_dataset(scene_in = "modular02a", data_path = "\\training_data\\2023_14_11"):
    # Defining usefull variables
    path = os.getcwd() + data_path
    print(path)
    file_list = os.listdir(path)
    scene_files_list = [item for item in file_list if scene_in in item and "pkl" in item]


    file = open(path + "\\" + scene_files_list[0], "rb")
    training_data = pickle.load(file)
    training_df = pd.DataFrame(training_data)

    #List to store the name for every joint data column
    increments_columns = []
    prev_j_positions_columns = []
    post_j_positions_columns = []

    #Creates the name for every column
    num_joints = len(training_data[-1]["increments"])
    for joint_n in range(num_joints):
        #List to split inputs per joint
        joint_inc_col_name = "increments_" + str(joint_n)
        increments_columns.append(joint_inc_col_name)

        prev_joint_pos_col_name = "prev_j_positions_" + str(joint_n)
        prev_j_positions_columns.append(prev_joint_pos_col_name)

        #List to split outputs per joint
        post_joint_pos_col_name = "post_j_positions_" + str(joint_n)
        post_j_positions_columns.append(post_joint_pos_col_name)


    #Input columns per joint
    increments_df = pd.DataFrame(training_df['increments'].to_list())
    increments_df.columns = increments_columns


    prev_j_positions_df = pd.DataFrame(training_df['prev_j_positions'].to_list())
    prev_j_positions_df.columns = prev_j_positions_columns


    #Builds the X dataframe
    X_df = pd.concat([increments_df, prev_j_positions_df, 
                    training_df["prev_pos_x"], training_df["prev_pos_y"], training_df["prev_pos_z"]], 
                    axis="columns")


    #Output columns per joint
    post_j_positions_df = pd.DataFrame(training_df['post_j_positions'].to_list())
    post_j_positions_df.columns = post_j_positions_columns


    #Builds the y dataframe
    y_df = pd.concat([post_j_positions_df, 
                    training_df["post_pos_x"], training_df["post_pos_y"], training_df["post_pos_z"]], 
                    axis="columns")

    return(X_df,y_df)

def adapt_X_y(X_a,y_a):
    X_a_cols = ['current_state_x', 'current_state_y', 'current_state_z', 'joint0_current_state_rad', 'joint1_current_state_rad', 'joint0_actions_rad', 'joint1_actions_rad']

    X_a = pd.concat([X_a["prev_pos_x"], X_a["prev_pos_y"] , X_a["prev_pos_z"],
                X_a["prev_j_positions_0"], X_a["prev_j_positions_1"],
                X_a["increments_0"], X_a["increments_1"]], 
                        axis="columns")

    X_a.columns = X_a_cols

    y_a_cols = ['future_state_x', 'future_state_y', 'future_state_z', 'joint0_future_state_rad', 'joint1_future_state_rad']

    y_a = pd.concat([y_a["post_pos_x"], y_a["post_pos_y"] , y_a["post_pos_z"],
                y_a["post_j_positions_0"], y_a["post_j_positions_1"]], 
                        axis="columns")
    y_a.columns = y_a_cols
    return X_a, y_a

def plot_history(history, loss):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel(f'{loss} Loss')
  plt.plot(hist['epoch'], hist['loss'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_loss'],
           label = 'Val Error')
  #plt.ylim([0,0.1])
  plt.legend()
  plt.show()

def plot_pred_vs_test(test, pred):
    columns = test.columns.tolist()
    max_list = []
    min_list = []
    mse_list = []
    perc_mse_list = []
    for column_num in range(len(columns)):
        plt.figure()
        plt.scatter(test.iloc[:,column_num], pred.iloc[:,column_num], s=1)

        #Getting important values
        minim = min(test.iloc[:,column_num])
        min_list.append(minim)
        maxim = max(test.iloc[:,column_num])
        max_list.append(maxim)
        mse = mean_squared_error(test.iloc[:,column_num], pred.iloc[:,column_num])
        mse_list.append(round(mse,6))
        rang = (maxim-minim)
        perc = mse/rang *100
        perc_mse_list.append(round(perc,6))
        title = columns[column_num] + " - mse: " + str(round(mse,6))
        plt.title(title)
        plt.xlabel('True Values')
        plt.ylabel('Predictions')
        # Para tener una linea recta con la cual comparar los valores y que no altere
        # los limites de la figura se grafican los valores reales con ellos mismos
        plt.plot(test.iloc[:,column_num],test.iloc[:,column_num])
        plt.grid()
    data = {"Perception": columns, "Max.": max_list, "Min.": min_list, "Mse": mse_list, "Mse perc.": perc_mse_list}
    data_df = pd.DataFrame(data)
    return(data_df)

def export2path(scene, model, scaler):
    timestr = time.strftime("_%Y_%d_%m")
    models_path = "models\\" + timestr[1:]
    if not os.path.exists(models_path):
        os.mkdir(models_path)
        print(f"Directorio '{models_path}' ha sido creado.")
    else:
        print(f"El directorio '{models_path}' ya existe.")
    model.save(models_path + "\\model_" + scene + timestr + ".keras")
    with open(models_path + "\\scaler_" + scene + timestr + '.pkl', 'wb') as file:
        pickle.dump(scaler, file)

In [26]:
def export(scene, model, scaler):
    model.save("model_" + scene + ".keras")
    with open("scaler_" + scene + '.pkl', 'wb') as file:
        pickle.dump(scaler, file)
    with open("model_" + scene + '.pkl', 'wb') as file:
        pickle.dump(model, file)

In [21]:
scene = "modular02"
data_path = ""
X, y =load_dataset(scene, data_path)

# Se separan los datos de entrenmiento (80%) y los de prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(
X, y,
test_size=0.2, random_state=42)

# Scaling de los datos 
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled.shape

c:\Users\carlo\OneDrive\Imágenes\Documentos\GitHub\EMERGE\Morphology\Multimodular\modular02_learning


(28000, 7)

In [27]:
# Creacion y entrenamiento del modelo
model = Sequential()

# Capa de entrada (correspondiente al número de entradas)
model.add(Dense(units=7, input_dim=7, activation='relu'))

# Capas ocultas con 24 unidades cada una
model.add(Dense(units=24, activation='relu'))

# Capa de salida con 5 unidades (correspondiente al número de salidas) y activación lineal
model.add(Dense(units=5, activation='linear'))

# Compilar el modelo con una función de pérdida adecuada para regresión
model.compile(optimizer=Adam(learning_rate=0.001), loss=Huber(delta=1.0))

# Supongamos que X_train y y_train son tus datos de entrenamiento
history = model.fit(X_train_scaled, y_train, epochs=25, batch_size=32, validation_split=0.2)

export(scene, model, scaler)

Epoch 1/25
700/700 [==============================] - 1s 1ms/step - loss: 0.0397 - val_loss: 0.0023
Epoch 2/25
700/700 [==============================] - 1s 1ms/step - loss: 0.0013 - val_loss: 6.0070e-04
Epoch 3/25
700/700 [==============================] - 1s 1ms/step - loss: 3.8918e-04 - val_loss: 2.1891e-04
Epoch 4/25
700/700 [==============================] - 1s 1ms/step - loss: 1.6212e-04 - val_loss: 1.1074e-04
Epoch 5/25
700/700 [==============================] - 1s 1ms/step - loss: 1.0191e-04 - val_loss: 8.8626e-05
Epoch 6/25
700/700 [==============================] - 1s 1ms/step - loss: 7.9960e-05 - val_loss: 6.8723e-05
Epoch 7/25
700/700 [==============================] - 1s 1ms/step - loss: 6.7557e-05 - val_loss: 5.7086e-05
Epoch 8/25
700/700 [==============================] - 1s 1ms/step - loss: 6.4054e-05 - val_loss: 5.8213e-05
Epoch 9/25
700/700 [==============================] - 1s 1ms/step - loss: 5.8045e-05 - val_loss: 6.1431e-05
Epoch 10/25
700/700 [===================

In [28]:
def main():
    scene = "modular02"
    data_path = ""
    X, y =load_dataset(scene, data_path)
    
    # Se separan los datos de entrenmiento (80%) y los de prueba (20%)
    X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2, random_state=42)

    # Scaling de los datos 
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_train_scaled.shape

    # Creacion y entrenamiento del modelo
    model = Sequential()

    # Capa de entrada (correspondiente al número de entradas)
    model.add(Dense(units=7, input_dim=7, activation='relu'))

    # Capas ocultas con 24 unidades cada una
    model.add(Dense(units=24, activation='relu'))

    # Capa de salida con 5 unidades (correspondiente al número de salidas) y activación lineal
    model.add(Dense(units=5, activation='linear'))

    # Compilar el modelo con una función de pérdida adecuada para regresión
    model.compile(optimizer=Adam(learning_rate=0.001), loss=Huber(delta=1.0))

    # Supongamos que X_train y y_train son tus datos de entrenamiento
    history = model.fit(X_train_scaled, y_train, epochs=25, batch_size=32, validation_split=0.2)

    export(scene, model, scaler)

In [29]:
main()

c:\Users\carlo\OneDrive\Imágenes\Documentos\GitHub\EMERGE\Morphology\Multimodular\modular02_learning
Epoch 1/25
700/700 [==============================] - 1s 1ms/step - loss: 0.0488 - val_loss: 0.0052
Epoch 2/25
700/700 [==============================] - 1s 1ms/step - loss: 0.0020 - val_loss: 0.0010
Epoch 3/25
700/700 [==============================] - 1s 1ms/step - loss: 7.7461e-04 - val_loss: 5.9815e-04
Epoch 4/25
700/700 [==============================] - 1s 1ms/step - loss: 4.9653e-04 - val_loss: 4.0054e-04
Epoch 5/25
700/700 [==============================] - 1s 1ms/step - loss: 3.4559e-04 - val_loss: 2.9539e-04
Epoch 6/25
700/700 [==============================] - 1s 1ms/step - loss: 2.4399e-04 - val_loss: 2.2541e-04
Epoch 7/25
700/700 [==============================] - 1s 1ms/step - loss: 1.6814e-04 - val_loss: 1.4331e-04
Epoch 8/25
700/700 [==============================] - 1s 1ms/step - loss: 1.2028e-04 - val_loss: 1.0965e-04
Epoch 9/25
700/700 [==============================]